<a href="https://colab.research.google.com/github/Amsterdam-Internships/Readability-Lexical-Simplification/blob/master/Combined_Loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 6.5 MB 37.5 MB/s 
     |████████████████████████████████| 596 kB 36.3 MB/s 
     |████████████████████████████████| 67 kB 4.1 MB/s 
     |████████████████████████████████| 895 kB 43.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import random
import torch

In [ ]:
normal = pd.read_csv("/content/drive/MyDrive/Thesis/notebooks/data/normal.aligned", sep = "\t", names=["subject","nr", "sentence"])
simple = pd.read_csv("/content/drive/MyDrive/Thesis/notebooks/data/simple.aligned", sep = "\t", names=["subject","nr", "sentence"])

In [ ]:
normal[:100]

,subject,nr,sentence
0,"Cherokee, Oklahoma",0,It is the county seat of Alfalfa County .
1,"Cherokee, Oklahoma",0,"Cherokee is a city in Alfalfa County , Oklahom..."
2,Skateboard,5,Skateboard decks are usually between 28 and 33...
3,Skateboard,5,The underside of the deck can be printed with ...
4,Skateboard,6,This was created by two surfers ; Ben Whatson ...
...,...,...,...
95,Invisible ink,2,The ink is later made visible by different met...
96,Invisible ink,3,One can obtain toy invisible ink pens which ha...
97,Lizard,3,Some species of lizard also utilize bright col...
98,Lizard,4,The particular innovation in this respect is t...


In [ ]:
normal_sents = normal['sentence'].tolist()
simple_sents = simple['sentence'].tolist()

In [ ]:
normal_sents = normal_sents[:1000]
simple_sents = simple_sents[:1000]

In [ ]:
normal_sents[:5]

['It is the county seat of Alfalfa County .',
 'Cherokee is a city in Alfalfa County , Oklahoma , United States .',
 'Skateboard decks are usually between 28 and 33 inches long .',
 'The underside of the deck can be printed with a design by the manufacturer , blank , or decorated by any other means .',
 'This was created by two surfers ; Ben Whatson and Jonny Drapper .']

In [ ]:
simple_sents[:5]

['It is the county seat of Alfalfa County .',
 'Cherokee is a city of Oklahoma in the United States .',
 'Skateboard decks are normally between 28 and 33 inches long .',
 'The bottom of the deck can be printed with a design by the maker . Or it can be blank .',
 'The longboard was made by two surfers ; Ben Whatson and Jonny Drapper .']

In [ ]:
# . I ultimately had one document which contained multiple paragraphs and each paragraph pertained to one topic. 
# I’m using each paragraph as a separate context when using extractive QA. 
# This ultimately gave me a list of strings where each string was a paragraph (ie. multiple sentences). 
# This was stored inside the ‘text’ variable.

sentence_a = []
sentence_b = []
label = []

# Task goal: direction of simplification

# bag_of_sentences = [sentence.strip() for paragraph in text for sentence in paragraph.split('.') if sentence != '']
# bag_size = len(bag_of_sentences)


for normal_sent, simple_sent in zip(normal_sents, simple_sents):
  if random.random()>0.5:
    sentence_a.append(normal_sent)
    sentence_b.append(simple_sent)
    label.append(1)
  else:
    sentence_a.append(simple_sent)
    sentence_b.append(normal_sent)
    label.append(0)

from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-large-uncased-whole-word-masking')
inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', 
                   max_length=250, truncation=True, padding='max_length')



Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Creating labels for NSP
inputs['next_sentence_label'] = torch.LongTensor([label]).T

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()
rand = torch.rand(inputs.input_ids.shape)
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102) * (inputs.input_ids != 0)

selection = []
for i in range(inputs.input_ids.shape[0]):
  selection.append(torch.flatten(mask_arr[i].nonzero()).tolist())

for i in range(inputs.input_ids.shape[0]):
  inputs.input_ids[i, selection[i]] = 103


In [ ]:
from transformers import BertForPreTraining

In [ ]:
class OurDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)
dataset = OurDataset(inputs)
loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=False)

In [ ]:
import torch.optim as optim

In [ ]:
model = BertForPreTraining.from_pretrained('bert-large-uncased-whole-word-masking')

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
device = torch.device('cpu')
model.to(device)

BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [ ]:
model.train()

BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [ ]:
from transformers import AdamW

optim = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=False)
    for batch in loop:
        print("batch")
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        print("after input to device")
        token_type_ids = batch['token_type_ids'].to(device)
        print("after token_type_ids to device")
        attention_mask = batch['attention_mask'].to(device)
        print("after attention_mask to device")
        
        next_sentence_label = batch['next_sentence_label'].to(device)
        print("after NSL to device")
        
        labels = batch['labels'].to(device)
        print("after labels to device")
        
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        next_sentence_label=next_sentence_label,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())


  0%|          | 0/125 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


batch
after input to device
after token_type_ids to device
after attention_mask to device
after NSL to device
after labels to device


Epoch 0:   1%|          | 1/125 [01:08<2:21:51, 68.64s/it, loss=29.7]

batch
after input to device
after token_type_ids to device
after attention_mask to device
after NSL to device
after labels to device


Epoch 0:   2%|▏         | 2/125 [02:11<2:13:22, 65.06s/it, loss=22.3]

...

batch
after input to device
after token_type_ids to device
after attention_mask to device
after NSL to device
after labels to device


Epoch 1:  98%|█████████▊| 123/125 [2:00:56<01:58, 59.23s/it, loss=0.356]

batch
after input to device
after token_type_ids to device
after attention_mask to device
after NSL to device
after labels to device


Epoch 1:  99%|█████████▉| 124/125 [2:01:53<00:58, 58.70s/it, loss=0.51]

batch
after input to device
after token_type_ids to device
after attention_mask to device
after NSL to device
after labels to device


In [ ]:
def predict_masked_sent(text, top_k, model, tokenizer):
    model.to(device)

    model.eval()

    # Tokenize input
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    # tokens_tensor = tokens_tensor.to('cuda')

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
        token_weight = top_k_weights[i]
        print("[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))

In [ ]:
sentence = "[CLS] we [MASK] documents based on subject. [SEP] we cluster documents based on subject [CLS]"
sentence2 = "[CLS] we [MASK] documents based on subject. [CLS]"

predict_masked_sent(sentence, 5, model, tokenizer)
print()
predict_masked_sent(sentence2, 5, model, tokenizer)


[MASK]: 'cluster'  | weights: 0.43790990114212036
[MASK]: '[PAD]'  | weights: 0.1074443832039833
[MASK]: 'add'  | weights: 0.011403925716876984
[MASK]: 'clustered'  | weights: 0.010327151045203209
[MASK]: 'compact'  | weights: 0.009970532730221748

[MASK]: '##ve'  | weights: 0.17298102378845215
[MASK]: 'have'  | weights: 0.04960228130221367
[MASK]: 'to'  | weights: 0.04892673343420029
[MASK]: 'was'  | weights: 0.044326528906822205
[MASK]: 'had'  | weights: 0.03716942295432091


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [ ]:
base_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
base_model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
predict_masked_sent(sentence, 5, base_model, base_tokenizer)
print()
predict_masked_sent(sentence2, 5, base_model, base_tokenizer)

[MASK]: 'cluster'  | weights: 0.33416369557380676
[MASK]: 'compare'  | weights: 0.16992069780826569
[MASK]: 'label'  | weights: 0.03357220068573952
[MASK]: 'create'  | weights: 0.023759303614497185
[MASK]: 'construct'  | weights: 0.021330634132027626

[MASK]: 'have'  | weights: 0.05249989032745361
[MASK]: 'are'  | weights: 0.037207696586847305
[MASK]: 'do'  | weights: 0.02007931098341942
[MASK]: '.'  | weights: 0.015799084678292274
[MASK]: 'were'  | weights: 0.014494902454316616


In [ ]:
import os

In [ ]:
output_dir = '/content/drive/MyDrive/Thesis/notebooks/23march'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to /content/drive/MyDrive/Thesis/notebooks/23march


('/content/drive/MyDrive/Thesis/notebooks/23march/tokenizer_config.json',
 '/content/drive/MyDrive/Thesis/notebooks/23march/special_tokens_map.json',
 '/content/drive/MyDrive/Thesis/notebooks/23march/vocab.txt',
 '/content/drive/MyDrive/Thesis/notebooks/23march/added_tokens.json',
 '/content/drive/MyDrive/Thesis/notebooks/23march/tokenizer.json')

In [ ]:
from transformers import BertModel, BertTokenizer, AutoModel, AutoTokenizer

In [ ]:
loaded_model = BertForPreTraining.from_pretrained("/content/drive/MyDrive/Thesis/notebooks/23march")
loaded_tokenizer = BertTokenizerFast.from_pretrained("/content/drive/MyDrive/Thesis/notebooks/23march")

In [ ]:
predict_masked_sent(sentence, 5, loaded_model, loaded_tokenizer)
predict_masked_sent(sentence2, 5, loaded_model, loaded_tokenizer)

[MASK]: 'cluster'  | weights: 0.43790990114212036
[MASK]: '[PAD]'  | weights: 0.1074443832039833
[MASK]: 'add'  | weights: 0.011403925716876984
[MASK]: 'clustered'  | weights: 0.010327151045203209
[MASK]: 'compact'  | weights: 0.009970532730221748
[MASK]: '##ve'  | weights: 0.17298102378845215
[MASK]: 'have'  | weights: 0.04960228130221367
[MASK]: 'to'  | weights: 0.04892673343420029
[MASK]: 'was'  | weights: 0.044326528906822205
[MASK]: 'had'  | weights: 0.03716942295432091
